#### Consider the linear SDE:

$$ \mathrm{d}X(t) = \mu X(t) \mathrm{d}t + \sigma X(t) \mathrm{d}W(t), X(0) = X_0$$ 

#### where $\mu, \sigma$ are real constants.
#### The exact solution to this SDE is

$$ X(t) = X(0) \exp \left ( (\mu - \frac{1}{2} \sigma^2) t + \sigma W(t) \right )$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(100)

In [ ]:
mu = 0.5
sigma = 6**0.5
X_0 = 1
T = 100
N = 2**10

dt = float(T) / N
t = np.linspace(0, T, N+1)


In [ ]:
def EM_solver(R, dt, dW, N, X_ref):
    Dt = R * dt
    L = N // R
    X_approx = np.zeros(L + 1)
    X_approx[0] = X_ref[0]

    for j in range(1, L+1):
        W_acc = np.sum(dW[0][range(R*(j-1), R*j)])
        X_approx[j] = X_approx[j-1] + mu * X_approx[j-1] * Dt + sigma * X_approx[j-1] * W_acc

    X_L = X_approx[-1]
    X_T = X_ref[-1]
    
    err = np.abs(X_L - X_T)   
    # err = np.abs(X_approx - X_ref[::R])   
    # print("Error: {:.4e}".format(err))
    
    return err, X_approx, X_L, X_T
    

In [ ]:
def order(R, X_0 = X_0, dt = dt, N = N, solver = EM_solver):

    dW = np.sqrt(dt) * np.random.randn(1, N)
    W = np.cumsum(dW)

    X_ref = X_0 * np.exp((mu - 0.5*sigma**2)*t[1:] + sigma*W)
    X_ref = np.insert(X_ref, obj = 0, values = X_0)

    err, X_approx, X_L, X_T = solver(R, dt, dW, N, X_ref)

    return err, X_approx, X_L, X_T


In [ ]:
MC = 1

In [ ]:
R_1 = 2**1
sol_list = np.zeros((MC, N // R_1 + 1))
for i in range(MC):
    _, X_approx, X_L, X_T = order(R = R_1, X_0 = X_0, dt = dt, N = N, solver = EM_solver)
    sol_list[i, :] = np.abs(X_approx)
    
Exception_sol1_case1 = np.mean(sol_list, axis = 0)

In [ ]:
R_2 = 2**3
sol_list = np.zeros((MC, N // R_2 + 1))
for i in range(MC):
    _, X_approx, X_L, X_T = order(R = R_2, X_0 = X_0, dt = dt, N = N, solver = EM_solver)
    sol_list[i, :] = np.abs(X_approx)
    
Exception_sol1_case2 = np.mean(sol_list, axis = 0)

In [ ]:
R_3 = 2**5
sol_list = np.zeros((MC, N // R_3 + 1))
for i in range(MC):
    _, X_approx, X_L, X_T = order(R = R_3, X_0 = X_0, dt = dt, N = N, solver = EM_solver)
    sol_list[i, :] = np.abs(X_approx)
    
Exception_sol1_case3 = np.mean(sol_list, axis = 0)

In [ ]:
# test on for MC paths

L_1 = N // R_1
ax = plt.subplot(111)
ax.plot(np.linspace(0, T, L_1+1), np.log(Exception_sol1_case1), "r-x")
L_2 = N // R_2
ax.plot(np.linspace(0, T, L_2+1), np.log(Exception_sol1_case2), "b-+")
L_3 = N // R_3
ax.plot(np.linspace(0, T, L_3+1), np.log(Exception_sol1_case3), "g-*")

ax.grid()
ax.legend(("case 1","case 2", "case 3"), loc = "best")
plt.savefig("./asym_stable.pdf")
plt.show()